# Dimensionality Reduction
On this notebook we will compare different techniques for dimensionality reduction (and clustering)
* PCA
* t-sne
* umap

#### Use case (scRNA-seq)
The ability to isolate and sequence the genetic material of single cells allows researchers to identify which genes are active in each cell.

#### References
* https://towardsdatascience.com/tsne-vs-umap-global-structure-4d8045acba17
* https://towardsdatascience.com/guide-to-dimensionality-reduction-in-single-cell-rna-seq-analysis-1d77284eed1c
* [Original Implementation](https://github.com/CrazyDaffodils/scRNA)

In [2]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
import seaborn as sns
import pandas as pd

#### Get Dataset

#### Use Principal Components Analysys (PCA)

In [3]:
pca = PCA(n_components = 2).fit_transform(X_std)
pca_df = pd.DataFrame(data=pca, columns=['PC1','PC2']).join(labels)
palette = sns.color_palette("muted", n_colors=5)
sns.set_style("white")
sns.scatterplot(x='PC1',y='PC2',hue='Class',data=pca_df, palette=palette, linewidth=0.2, s=30, alpha=1).set_title('PCA')

NameError: name 'X_std' is not defined

#### Use T-sne

In [4]:
model = TSNE(learning_rate = 10, n_components = 2, random_state=123, perplexity = 30)
tsne = model.fit_transform(X_std)
tsne_df = pd.DataFrame(data=tsne, columns=['t-SNE1','t-SNE2']).join(labels)
palette = sns.color_palette("muted", n_colors=5)
sns.set_style("white")
sns.scatterplot(x='t-SNE1',y='t-SNE2',hue='Class',data=tsne_df, palette=palette, linewidth=0.2, s=30, alpha=1).set_title('t-SNE')

NameError: name 'X_std' is not defined

#### Use UMAP

In [5]:
model = UMAP(n_neighbors = 40, min_dist = 0.4, n_components = 2)
umap = model.fit_transform(X_std)
umap_df = pd.DataFrame(data=umap, columns=['UMAP1','UMAP2']).join(labels)
palette = sns.color_palette("muted", n_colors=5)
sns.set_style("white")
sns.scatterplot(x='UMAP1',y='UMAP2',hue='Class',data=umap_df, palette=palette, linewidth=0.2, s=30, alpha=1).set_title('UMAP')

NameError: name 'X_std' is not defined